# <a id="top">Lab 2: Response Level Detection</a>

## Detecting LLM Hallucinations Through Response Consistency Analysis

For many applications built on closed-source, proprietary models accessed via APIs, we don't have access to the model's internal states (like weights or logits) or ground truth data to compare against. **Response Level Detection** techniques are designed for this scenario, treating the model as an opaque system and relying only on its input-output behavior to assess reliability and detect potential hallucinations.

In this notebook, we will implement and compare three cutting-edge response level detection methods that work without requiring reference answers or model internals. These methods analyze the **consistency and coherence of multiple model responses** to the same prompt.

We'll implement and compare three cutting-edge detection methods:
1. **Semantic Similarity Analysis** - Measuring consistency across multiple model responses using embeddings
2. **Non-Contradiction Probability** - Using Natural Language Inference (NLI) to detect logical contradictions
3. **Normalized Semantic Negentropy (NSN)** - Measuring semantic uncertainty through response clustering

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>💡 Key Learning Objectives</h4>
    <ul>
        <li>Understand Response Level hallucination detection techniques</li>
        <li>Learn how to implement semantic similarity analysis with sentence embeddings</li>
        <li>Master contradiction detection using Natural Language Inference (NLI) models</li>
        <li>Apply semantic entropy methods for uncertainty quantification</li>
        <li>Build AI agents with Strands Agents SDK that automatically detect hallucinations</li>
    </ul>
</div>
<br/>

## Use-Case Overview

Response Level hallucination detection is crucial when:
- You don't have ground truth answers to compare against
- You need to assess model reliability in production without access to model internals
- You want to identify when a model is "making things up"
- You need to implement confidence scoring for AI responses

Our approach leverages the principle that **reliable knowledge should be consistent** across multiple generations with different sampling parameters, while hallucinations tend to vary significantly.


## Sections

This notebook has the following sections:

1. [Environment Setup and Model Configuration](#1.-Environment-Setup-and-Model-Configuration)
2. [Method 1: Semantic Similarity Detection](#2.-Method-1:-Semantic-Similarity-Detection)
3. [Method 2: Non-Contradiction Probability](#3.-Method-2:-Non-Contradiction-Probability)
4. [Method 3: Normalized Semantic Negentropy (NSN)](#4.-Method-3:-Normalized-Semantic-Negentropy-(NSN))
5. [Comparative Analysis Across Models](#5.-Comparative-Analysis-Across-Models)
6. [Building Production-Ready Agents with Semantic Similarity](#6.-Building-Production-Ready-Agents-with-Semantic-Similarity)
7. [Conclusion and Best Practices](#7.-Conclusion-and-Best-Practices)
8. [(Optional) Challenge Exercises](#8.-(Optional)-Challenge-Exercises)
    
Please work from top to bottom and don't skip sections as this could lead to error messages due to missing dependencies.

----

## 1. Environment Setup and Model Configuration
(<a href="#top">Go to top</a>)

**If you haven't already** installed the workshop's dependencies (from [pyproject.toml](./pyproject.toml)), you can un-comment (remove `# `) and run the below cell to do so. We've commented it out by default, assuming you already ran it at the start of lab 0:

In [ ]:
# %pip install -q -e .

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; padding-top: 15px;">
    <h4>🔄 Restart the kernel after installing</h4>
    <p>
        <strong>IF</strong> you ran the above install command cell, you'll need to restart the
        notebook kernel afterwards for the installations to take full effect.
    </p>
    <p>
        Note that you may see some error notices about dependency conflicts in SageMaker Studio
        environments, but this is okay as long as the installations are completed.
    </p>
</div>
<br/>

With the installation complete, you're ready to import the libraries we'll use in the notebook:

In [ ]:
import boto3
import json
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import time
from typing import List, Dict, Tuple
import warnings

warnings.filterwarnings("ignore")

print("All packages imported successfully!")

In [ ]:
# Retrieve stored endpoint configuration
%store -r endpoint_name
%store -r inference_component_name

try:
    endpoint_name, inference_component_name
except NameError as e:
    raise RuntimeError(
        "SageMaker endpoint not found. Please check you've run lab 0 first!"
    )

### Configure AWS Bedrock and SageMaker Models

We'll test our hallucination detection methods across multiple models available on AWS Bedrock and SageMaker. This allows us to compare how different models perform in terms of consistency and reliability.

For SageMaker endpoints, we'll automatically discover available endpoints with inference components that you've deployed.

In [ ]:
# Initialize Bedrock and SageMaker clients
bedrock_client = boto3.client("bedrock-runtime")
sagemaker_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

# Available models on Bedrock
BEDROCK_MODELS = {"claude-haiku-4-5": "global.anthropic.claude-haiku-4-5-20251001-v1:0"}

print(f"✓ Configured {len(BEDROCK_MODELS)} Bedrock model(s) for testing:")
for name in BEDROCK_MODELS.keys():
    print(f"  • {name}")
if endpoint_name:
    print(f"- SageMaker Endpoint: {endpoint_name}")
    print(f"- Inference Component: {inference_component_name}")
    sagemaker_endpoint = f"{endpoint_name},{inference_component_name}"
else:
    print(f"\n No SageMaker endpoints with inference components found.")

### Model Response Generation Function

This function handles the different API formats for various Bedrock models, allowing us to generate responses consistently across all model types.

In [ ]:
def generate_bedrock_response(
    prompt: str, model_key: str, temperature: float = 0.7
) -> str:
    """
    Generate a response from a Bedrock model with proper error handling.

    Args:
        prompt: The input prompt to send to the model
        model_key: Key from BEDROCK_MODELS dictionary
        temperature: Sampling temperature

    Returns:
        Generated text response or empty string on error
    """
    model_id = BEDROCK_MODELS[model_key]

    try:
        # Prepare the conversation message format for Converse API
        messages = [{"role": "user", "content": [{"text": prompt}]}]

        # Set up inference configuration parameters
        inference_config = {"maxTokens": 200, "temperature": temperature}

        # Use the Converse API for all model types
        response = bedrock_client.converse(
            modelId=model_id, messages=messages, inferenceConfig=inference_config
        )

        # Extract the response text from the Converse API response
        return response["output"]["message"]["content"][0]["text"]

    except Exception as e:
        print(f"Error with {model_key}: {str(e)}")
        return ""


# Test the function with a simple prompt
test_response = generate_bedrock_response("What is 2+2?", "claude-haiku-4-5", 0.1)
if test_response:
    print(f"Model connection successful! Test response: {test_response[:50]}...")
else:
    print("Model connection failed. Please check your model IDs and region.")

This function handles the invocation to the models deployed to the SageMaker AI real-time endpoints.

In [ ]:
def generate_sagemaker_response(
    prompt: str, endpoint_name: str, inference_component: str, temperature: float = 0.7
) -> str:
    """
    Generate a response from a SageMaker endpoint with proper error handling.

    Args:
        prompt: The input prompt to send to the model
        endpoint_name: Name of the SageMaker endpoint
        temperature: Sampling temperature

    Returns:
        Generated text response or empty string on error
    """
    try:
        # Prepare the payload for SageMaker endpoint
        payload = {
            "messages": [{"role": "user", "content": prompt}],
            "temperature": temperature,
            "top_p": 0.9,
        }

        # Invoke the SageMaker endpoint
        response = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            InferenceComponentName=inference_component,
            ContentType="application/json",
            Body=json.dumps(payload),
        )

        # Parse the response
        result = json.loads(response["Body"].read().decode())

        # Extract the response text (adjust based on your model's response format)
        if "choices" in result and len(result["choices"]) > 0:
            return result["choices"][0]["message"]["content"]
        elif "generated_text" in result:
            return result["generated_text"]
        else:
            print(f"Unexpected response format: {result}")
            return ""

    except Exception as e:
        print(f"Error with SageMaker endpoint {endpoint_name}: {str(e)}")
        return ""


# Test the function with a simple prompt
test_response = generate_sagemaker_response(
    "What is 2+2?", endpoint_name, inference_component_name, 0.1
)
if test_response:
    print(f"Model connection successful! Test response: {test_response[:50]}...")
else:
    print("Model connection failed. Please check your model IDs and region.")

Create a unified response generation function:

In [ ]:
def generate_response(
    prompt: str,
    model_identifier: str,
    temperature: float = 0.7,
    invocation_method: str = "bedrock",
) -> str:
    """
    Unified function to generate responses from either Bedrock or SageMaker.

    Args:
        prompt: The input prompt to send to the model
        model_identifier: Either model_key (for Bedrock) or endpoint_name (for SageMaker)
        temperature: Sampling temperature
        invocation_method: "bedrock" or "sagemaker"

    Returns:
        Generated text response or empty string on error
    """
    if invocation_method.lower() == "bedrock":
        return generate_bedrock_response(prompt, model_identifier, temperature)
    elif invocation_method.lower() == "sagemaker":
        model_identifier, inference_component = model_identifier.split(",")
        return generate_sagemaker_response(
            prompt, model_identifier, inference_component, temperature
        )
    else:
        print(
            f"Invalid invocation method: {invocation_method}. Use 'bedrock' or 'sagemaker'"
        )
        return ""

## 2. Method 1: Semantic Similarity Detection
(<a href="#top">Go to top</a>)

### How It Works

Semantic Similarity Detection works on the principle that **reliable information should be expressed consistently**, even when generated multiple times with different parameters.

**The Process:**
1. **Generate Multiple Responses**: Ask the same question several times with varying temperatures
2. **Create Embeddings**: Convert each response to a high-dimensional vector representation
3. **Calculate Similarities**: Measure cosine similarity between all response pairs
4. **Assess Consistency**: High similarity = reliable, low similarity = potential hallucination

**Key Principle**: Reliable information should be expressed consistently, regardless of sampling parameters.

In [ ]:
def semantic_similarity_detection(
    prompt: str,
    model_identifier: str = None,
    num_samples: int = 5,
    invocation_method: str = "bedrock",
) -> Dict:
    """
    Detect hallucinations using semantic similarity across multiple responses.

    Args:
        prompt: Question to ask the model
        model_identifier: Model key (for Bedrock) or endpoint name (for SageMaker)
        num_samples: Number of responses to generate and compare
        invocation_method: "bedrock" or "sagemaker" - which service to use

    Returns:
        Dictionary with similarity metrics and confidence assessment
    """

    print(f"🔍 SEMANTIC SIMILARITY DETECTION")
    print(f"Invocation Method: {invocation_method.upper()}")
    print(f"Model/Endpoint: {model_identifier}")
    print(f"Question: {prompt}")
    print("=" * 60)

    # Step 1: Generate multiple responses with different temperatures
    print(f"Generating {num_samples} responses using {invocation_method}...")
    responses = []
    for i in range(num_samples):
        # Use low temperature for first response, higher for others
        temp = 0.1 if i == 0 else 0.7
        response = generate_response(
            prompt,
            model_identifier,
            temperature=temp,
            invocation_method=invocation_method,
        )
        if response:
            responses.append(response)
            print(
                f"  Response {i + 1}: {response[:120]}{'...' if len(response) > 120 else ''}"
            )

    if len(responses) < 2:
        return {"error": "Need at least 2 responses for comparison"}

    # Step 2: Create semantic embeddings
    print(f"\nCreating semantic embeddings...")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(responses)

    # Step 3: Calculate pairwise similarities
    print(f"Calculating semantic similarities...")
    similarity_matrix = cosine_similarity(embeddings)

    # Get all pairwise similarities (excluding diagonal)
    mask = np.ones(similarity_matrix.shape, dtype=bool)
    np.fill_diagonal(mask, False)
    similarities = similarity_matrix[mask]

    # Step 4: Calculate metrics
    mean_similarity = np.mean(similarities)
    std_similarity = np.std(similarities)
    min_similarity = np.min(similarities)
    max_similarity = np.max(similarities)

    # Step 5: Determine confidence level
    if mean_similarity > 0.95 and std_similarity < 0.03:
        confidence = "🟢 HIGH"
        interpretation = "Responses are highly consistent - reliable knowledge"
    elif mean_similarity > 0.85 and std_similarity < 0.08:
        confidence = "🟡 MEDIUM"
        interpretation = "Moderate consistency - some uncertainty present"
    else:
        confidence = "🔴 LOW"
        interpretation = "Inconsistent responses - possible hallucination"

    # Display results
    print(f"\nRESULTS:")
    print(f"Mean Similarity:    {mean_similarity:.3f}")
    print(f"Std Deviation:      {std_similarity:.3f}")
    print(f"Min Similarity:     {min_similarity:.3f}")
    print(f"Max Similarity:     {max_similarity:.3f}")
    print(f"\nCONFIDENCE: {confidence}")
    print(f"Interpretation: {interpretation}")
    print("=" * 60)

    return {
        "method": "Semantic Similarity",
        "invocation_method": invocation_method,
        "model_identifier": model_identifier,
        "num_responses": len(responses),
        "mean_similarity": round(mean_similarity, 3),
        "std_similarity": round(std_similarity, 3),
        "min_similarity": round(min_similarity, 3),
        "confidence": confidence,
        "interpretation": interpretation,
        "responses": responses,
    }

### Test Semantic Similarity Detection

Let's test this method with a question that should have a clear, factual answer.

In [ ]:
result = semantic_similarity_detection(
    "What album was Dua Lipa's 'Hallucinate' featured on?",
    "claude-haiku-4-5",
    num_samples=5,
)

### Optional: Test with SageMaker Endpoint

If you have SageMaker endpoints with inference components available, you can test them here. Otherwise, skip to the next section.

In [ ]:
# Test with SageMaker endpoint (if available)
if sagemaker_endpoint:
    print(f"Testing with SageMaker endpoint: {sagemaker_endpoint}")
    result = semantic_similarity_detection(
        "What album was Dua Lipa's 'Hallucinate' featured on?",
        sagemaker_endpoint,
        num_samples=5,
        invocation_method="sagemaker",
    )
else:
    print("ℹ️  No SageMaker endpoints available - skipping this test.")
    print("   You can continue with Bedrock models only.")

## 3. Method 2: Non-Contradiction Probability
(<a href="#top">Go to top</a>)

### How It Works

Non-Contradiction Probability uses **Natural Language Inference (NLI)** to detect logical inconsistencies between multiple model responses.

**The Process:**
1. **Generate Multiple Responses**: Get several answers to the same question
2. **Pairwise Analysis**: Compare every response pair using an NLI model
3. **Contradiction Detection**: Identify logical contradictions between responses
4. **Probability Calculation**: Calculate the probability that responses don't contradict each other

**Key Principle**: Reliable knowledge should not contradict itself, while hallucinations often contain conflicting information.

In [ ]:
def non_contradiction_detection(
    prompt: str,
    model_identifier: str = None,
    num_samples: int = 5,
    invocation_method: str = "bedrock",
) -> Dict:
    """
    Detect hallucinations by analyzing logical contradictions between responses.

    Args:
        prompt: Question to ask the model
        model_identifier: Which model to test
        num_samples: Number of responses to generate and analyze
        invocation_method: "bedrock" or "sagemaker" - which service to use

    Returns:
        Dictionary with contradiction analysis and confidence assessment
    """

    print(f"NON-CONTRADICTION PROBABILITY DETECTION")
    print(f"Invocation Method: {invocation_method.upper()}")
    print(f"Model/Endpoint: {model_identifier}")
    print(f"Question: {prompt}")
    print("=" * 60)

    # Step 1: Generate multiple responses
    print(f"Generating {num_samples} responses...")
    responses = []
    for i in range(num_samples):
        temp = 0.1 if i == 0 else 0.7
        response = generate_response(
            prompt,
            model_identifier,
            temperature=temp,
            invocation_method=invocation_method,
        )
        if response:
            responses.append(response)
            print(
                f"  Response {i + 1}: {response[:120]}{'...' if len(response) > 120 else ''}"
            )

    if len(responses) < 2:
        return {"error": "Need at least 2 responses for analysis"}

    # Step 2: Load NLI model for contradiction detection
    print(f"\nLoading Natural Language Inference model...")
    nli_model = pipeline(
        "text-classification",
        model="microsoft/deberta-base-mnli",
        return_all_scores=True,
    )

    # Step 3: Analyze all response pairs for contradictions
    print(f"\nAnalyzing {len(responses)} responses for logical contradictions...")
    non_contradiction_scores = []
    pair_analyses = []

    for i in range(len(responses)):
        for j in range(i + 1, len(responses)):
            try:
                # Check both directions (A→B and B→A) for mutual contradiction
                result1 = nli_model(f"{responses[i]} [SEP] {responses[j]}")
                result2 = nli_model(f"{responses[j]} [SEP] {responses[i]}")

                # Extract contradiction probabilities
                contra_prob1 = 0.0
                contra_prob2 = 0.0

                # Parse NLI results
                if isinstance(result1, list) and len(result1) > 0:
                    inner_list1 = (
                        result1[0] if isinstance(result1[0], list) else result1
                    )
                    for item in inner_list1:
                        if (
                            isinstance(item, dict)
                            and item.get("label") == "CONTRADICTION"
                        ):
                            contra_prob1 = item.get("score", 0.0)
                            break

                if isinstance(result2, list) and len(result2) > 0:
                    inner_list2 = (
                        result2[0] if isinstance(result2[0], list) else result2
                    )
                    for item in inner_list2:
                        if (
                            isinstance(item, dict)
                            and item.get("label") == "CONTRADICTION"
                        ):
                            contra_prob2 = item.get("score", 0.0)
                            break

                # Calculate average contradiction probability
                avg_contradiction = (contra_prob1 + contra_prob2) / 2
                non_contradiction_score = 1 - avg_contradiction

                non_contradiction_scores.append(non_contradiction_score)
                pair_analyses.append(
                    {
                        "pair": f"{i + 1}-{j + 1}",
                        "contradiction": avg_contradiction,
                        "non_contradiction": non_contradiction_score,
                    }
                )

                print(
                    f"  Pair {i + 1}-{j + 1}: Contradiction={avg_contradiction:.3f}, Non-contradiction={non_contradiction_score:.3f}"
                )

            except Exception as e:
                print(f"  Error analyzing pair {i + 1}-{j + 1}: {str(e)}")
                non_contradiction_scores.append(0.5)  # Neutral score for errors

    if not non_contradiction_scores:
        return {"error": "Could not analyze any response pairs"}

    # Step 4: Calculate overall metrics
    mean_ncp = np.mean(non_contradiction_scores)
    std_ncp = np.std(non_contradiction_scores)
    min_ncp = np.min(non_contradiction_scores)

    # Step 5: Determine confidence level
    if mean_ncp > 0.8 and min_ncp > 0.7:
        confidence = "🟢 HIGH"
        interpretation = "No significant contradictions - facts are consistent"
    elif mean_ncp > 0.65 and min_ncp > 0.5:
        confidence = "🟡 MEDIUM"
        interpretation = "Minor contradictions detected - mostly consistent"
    else:
        confidence = "🔴 LOW"
        interpretation = "Significant contradictions detected - likely hallucination"

    print(f"\nRESULTS:")
    print(f"Mean Non-Contradiction: {mean_ncp:.3f}")
    print(f"Std Deviation:          {std_ncp:.3f}")
    print(f"Min Non-Contradiction:  {min_ncp:.3f}")
    print(f"\nCONFIDENCE: {confidence}")
    print(f"Interpretation: {interpretation}")
    print("=" * 60)

    return {
        "method": "Non-Contradiction Probability",
        "model": model_identifier,
        "confidence": confidence,
        "mean_ncp": mean_ncp,
        "std_ncp": std_ncp,
        "min_ncp": min_ncp,
        "interpretation": interpretation,
        "pair_analyses": pair_analyses,
        "responses": responses,
    }

### Test Non-Contradiction Detection

Let's test this method with the same question to see how it compares.

In [ ]:
result_ncp = non_contradiction_detection(
    "What album was Dua Lipa's 'Hallucinate' featured on?",
    "claude-haiku-4-5",
    num_samples=5,
)

## 4. Method 3: Normalized Semantic Negentropy (NSN)
(<a href="#top">Go to top</a>)

### How It Works

Normalized Semantic Negentropy is the most sophisticated method, measuring **semantic uncertainty** through response clustering.

**The Process:**
1. **Generate Multiple Responses**: Create diverse responses with different sampling parameters
2. **Semantic Clustering**: Group responses that convey the same meaning
3. **Entropy Calculation**: Measure how "scattered" the responses are across clusters
4. **Confidence Scoring**: Convert entropy to a confidence score (low entropy = high confidence)

**Key Principle**: Confident knowledge clusters tightly, while uncertain knowledge fragments across many different meanings.

In [ ]:
def semantic_entropy_detection(
    prompt: str,
    model_identifier: str = None,
    num_samples: int = 5,
    invocation_method: str = "bedrock",
) -> Dict:
    """
    Detect hallucinations using Normalized Semantic Negentropy (NSN).

    This method clusters responses by meaning and calculates semantic entropy
    to measure model uncertainty. Lower entropy indicates higher confidence.

    Args:
        prompt: Question to ask the model
        model_identifier: Which model to test
        num_samples: Number of responses to generate and cluster
        invocation_method: "bedrock" or "sagemaker" - which service to use

    Returns:
        Dictionary with entropy metrics and confidence assessment
    """

    print(f" NORMALIZED SEMANTIC NEGENTROPY DETECTION")
    print(f"Invocation Method: {invocation_method.upper()}")
    print(f"Model/Endpoint: {model_identifier}")
    print("=" * 60)

    # Step 1: Generate diverse responses
    print(f" Generating {num_samples} diverse responses...")
    responses = []
    for i in range(num_samples):
        temp = 0.1 if i == 0 else min(1.0, 0.5 + (i * 0.1))  # Cap at 1.0
        response = generate_response(
            prompt,
            model_identifier,
            temperature=temp,
            invocation_method=invocation_method,
        )
        if response:
            responses.append(response)
            print(
                f"  Response {i + 1}: {response[:120]}{'...' if len(response) > 120 else ''}"
            )

    if len(responses) < 3:
        return {"error": "Need at least 3 responses for clustering analysis"}

    # Step 2: Load NLI model for semantic clustering
    print(f"\nLoading NLI model for semantic clustering...")
    nli_model = pipeline(
        "text-classification",
        model="microsoft/deberta-base-mnli",
        return_all_scores=True,
    )

    # Step 3: Cluster responses by semantic meaning
    print(f"\n Clustering responses by semantic meaning...")
    clusters = []

    for i, response in enumerate(responses):
        assigned_to_cluster = False

        # Try to assign to existing cluster
        for cluster in clusters:
            # Compare with representative response from cluster
            representative = responses[cluster[0]]

            # Check for bidirectional entailment (same meaning)
            result1 = nli_model(f"{response} [SEP] {representative}")
            result2 = nli_model(f"{representative} [SEP] {response}")

            # Extract entailment scores
            entail1 = 0.0
            entail2 = 0.0

            if isinstance(result1, list) and len(result1) > 0:
                inner_list1 = result1[0] if isinstance(result1[0], list) else result1
                for item in inner_list1:
                    if isinstance(item, dict) and item.get("label") == "ENTAILMENT":
                        entail1 = item.get("score", 0.0)
                        break

            if isinstance(result2, list) and len(result2) > 0:
                inner_list2 = result2[0] if isinstance(result2[0], list) else result2
                for item in inner_list2:
                    if isinstance(item, dict) and item.get("label") == "ENTAILMENT":
                        entail2 = item.get("score", 0.0)
                        break

            # Use geometric mean for mutual entailment
            mutual_entailment = np.sqrt(entail1 * entail2)

            if mutual_entailment > 0.5:  # Threshold for "same meaning"
                cluster.append(i)
                assigned_to_cluster = True
                break

        # Create new cluster if no match found
        if not assigned_to_cluster:
            clusters.append([i])

    # Step 4: Display cluster information
    print(f"Found {len(clusters)} semantic clusters:")
    for i, cluster in enumerate(clusters):
        print(
            f"  Cluster {i + 1}: Responses {[j + 1 for j in cluster]} ({len(cluster)} responses)"
        )

    # Step 5: Calculate semantic entropy
    cluster_probabilities = [len(cluster) / len(responses) for cluster in clusters]
    semantic_entropy = -sum(p * np.log(p) for p in cluster_probabilities if p > 0)

    # Normalize entropy (0 = all same meaning, 1 = all different meanings)
    max_entropy = np.log(len(responses))
    normalized_entropy = semantic_entropy / max_entropy if max_entropy > 0 else 0
    nsn_score = 1 - normalized_entropy  # Convert to confidence score

    # Step 6: Calculate additional metrics
    fragmentation_ratio = len(clusters) / len(responses)
    largest_cluster_size = max(len(cluster) for cluster in clusters)
    cluster_dominance = largest_cluster_size / len(responses)

    # Step 7: Determine confidence level
    if nsn_score > 0.8 and fragmentation_ratio < 0.4:
        confidence = "🟢 HIGH"
        interpretation = "Responses cluster tightly - coherent understanding"
    elif nsn_score > 0.6 and fragmentation_ratio < 0.6:
        confidence = "🟡 MEDIUM"
        interpretation = "Moderate clustering - some uncertainty present"
    else:
        confidence = "🔴 LOW"
        interpretation = "Highly fragmented responses - model is uncertain"

    print(f"\n RESULTS:")
    print(f"Semantic Entropy:       {semantic_entropy:.3f}")
    print(f"NSN Score:              {nsn_score:.3f}")
    print(f"Fragmentation Ratio:    {fragmentation_ratio:.3f}")
    print(f"Cluster Dominance:      {cluster_dominance:.3f}")
    print(f"\n CONFIDENCE: {confidence}")
    print(f"\n Interpretation: {interpretation}")
    print("=" * 60)

    return {
        "method": "Normalized Semantic Negentropy",
        "model": model_identifier,
        "confidence": confidence,
        "nsn_score": nsn_score,
        "num_clusters": len(clusters),
        "semantic_entropy": semantic_entropy,
        "fragmentation_ratio": fragmentation_ratio,
        "cluster_dominance": cluster_dominance,
        "interpretation": interpretation,
        "clusters": clusters,
        "responses": responses,
    }

### Test Semantic Entropy Detection

Let's test this advanced method with our Mars question.

In [ ]:
result_nsn = semantic_entropy_detection(
    "What album was Dua Lipa's 'Hallucinate' featured on?",
    "claude-haiku-4-5",
    num_samples=5,
)

## 5. Comparative Analysis Across Models
(<a href="#top">Go to top</a>)

Now let's compare how different models perform across all three detection methods. This will help us understand which models are more reliable and which detection methods are most effective.

In [ ]:
def comprehensive_model_analysis(prompt: str, models_to_test: Dict = None) -> Dict:
    """
    Run all three detection methods across multiple models for comparison.

    Args:
        prompt: Question to test
        models_to_test: List of model keys to test (None = test all available)

    Returns:
        Comprehensive results dictionary
    """

    # if models_to_test is None:
    #     models_to_test = ['claude-haiku-4-5']

    print(f"COMPREHENSIVE MODEL ANALYSIS")
    print(f"Question: {prompt}")
    print(f"Models: {', '.join(models_to_test)}")
    print("=" * 80)

    results = {}
    methods = [
        ("Semantic Similarity", semantic_similarity_detection),
        ("Non-Contradiction", non_contradiction_detection),
        ("Semantic Entropy", semantic_entropy_detection),
    ]

    for model_key, model_values in models_to_test.items():
        invocation_method = model_key
        for model_name in model_values:
            model_results = {}

            for method_name, method_func in methods:
                try:
                    print(f"\n--- {method_name} ---")
                    result = method_func(
                        prompt,
                        model_name,
                        num_samples=4,
                        invocation_method=invocation_method,
                    )
                    model_results[method_name] = result
                    time.sleep(2)  # Rate limiting
                except Exception as e:
                    print(f"Error with {method_name}: {str(e)}")
                    model_results[method_name] = {"error": str(e)}

            results[model_key] = model_results

    return results


def display_comparison_summary(results: Dict):
    """
    Display a neat summary table of all results.
    """
    print(f"\n SUMMARY COMPARISON")
    print("=" * 80)
    print(f"{'Model':<15} {'Semantic Sim':<15} {'Non-Contra':<15} {'Sem Entropy':<15}")
    print("-" * 80)

    for model, model_results in results.items():
        row = f"{model:<15}"

        methods = ["Semantic Similarity", "Non-Contradiction", "Semantic Entropy"]
        for method in methods:
            if method in model_results and "confidence" in model_results[method]:
                confidence = model_results[method]["confidence"]
                # Extract just the confidence level (HIGH/MEDIUM/LOW)
                conf_level = confidence.split()[-1] if confidence else "ERROR"
                row += f"{conf_level:<15}"
            else:
                row += f"{'ERROR':<15}"

        print(row)

    print("=" * 80)

### Run Comprehensive Analysis

Let's test multiple models with different types of questions to see how they perform.

In [ ]:
# Test 1: Factual question with clear answer
print("TEST 1: Factual Question")

# Build models to test dictionary
models_to_test = {"bedrock": list(BEDROCK_MODELS.keys())}

# Add SageMaker endpoint if available
if sagemaker_endpoint:
    models_to_test["sagemaker"] = [sagemaker_endpoint]
    print(f"Testing Bedrock + SageMaker endpoint")
else:
    print(f"Testing {len(BEDROCK_MODELS)} Bedrock model(s) only")

results1 = comprehensive_model_analysis(
    "What album was Dua Lipa's 'Hallucinate' featured on?",
    models_to_test=models_to_test,
)
display_comparison_summary(results1)

## 6. Building Production-Ready Agents with Semantic Similarity
(<a href="#top">Go to top</a>)

In this section, we'll integrate response level hallucination detection into a production-ready agentic system using **AWS Strands Agents**. This demonstrates how to build agents that automatically detect and mitigate hallucinations in real-time.

### What are Strands Agents?

[Strands](https://github.com/strands-agents/sdk-python) is an open-source Python framework for building agentic AI applications. It provides:
- **Unified Model Interface**: Work with multiple LLM providers (Bedrock, SageMaker, etc.)
- **Tool Integration**: Connect agents to external tools and APIs
- **Observability**: Built-in tracing and monitoring via OpenTelemetry
- **Extensibility**: Easy to add custom behaviors like hallucination detection

### Our Approach: Semantic Similarity as a Guardrail

We'll build an agent that:
1. **Generates multiple candidate responses** in parallel for each user query
2. **Analyzes semantic similarity** between the candidates
3. **Intervenes if inconsistency is detected** (potential hallucination)
4. **Returns the most reliable response** or raises an alert

#### Why This Matters:

Unlike simple API wrappers, this pattern provides:
- **Automated Quality Control**: No need for human review of every response
- **Transparent Telemetry**: Full visibility into guardrail decisions via OpenTelemetry
- **Flexible Deployment**: Works with any Strands-compatible model (Bedrock, SageMaker, OpenAI, etc.)
- **Production-Ready**: Built-in error handling, logging, and monitoring


This pattern is ideal for production systems where you need automated quality control without human review of every response.

### Step 1: Prepare environment

In [ ]:
# Python Built-Ins:
import base64
import os

# External Dependencies:
from dotenv import load_dotenv
from strands import Agent
from strands.models.bedrock import BedrockModel
from strands.telemetry import StrandsTelemetry

# For our custom parallel model implementation:
import sys

sys.path.append("..")
from hallucination_utils.models.parallel import (
    ParallelModelWithConsolidation,
    ModelResponse,
    TraceAttributes,
)

print("✓ Imports successful!")
print(f"✓ Strands Agent module: {Agent.__module__}")

### Optional: Configure Langfuse for Observability

[Langfuse](https://langfuse.com/) provides production-grade observability for LLM applications. It integrates seamlessly with Strands via OpenTelemetry to track:
- Individual LLM calls and their responses
- Guardrail interventions and scores
- Token usage and latency metrics
- End-to-end conversation traces

The Langfuse configuration will be handled automatically by the `set_up_notebook_langfuse()` function below.

**Note**: You should have already obtained your Langfuse credentials following the **Enable Langfuse tracing** instruction in the Getting Started section. If you haven't set up Langfuse yet, you'll be prompted for your credentials when you run the cell below.

If you prefer not to use Langfuse, you can skip this setup - the agent will work fine without it, you just won't see traces in the Langfuse dashboard.

In [ ]:
# Import tracing utilities
from hallucination_utils.tracing import set_up_notebook_langfuse

# Set up Langfuse (will prompt for credentials if not already configured)
set_up_notebook_langfuse()

### Step 2: Create Base Model

Let's set up our base model - we'll use Claude 3.7 Sonnet via AWS Bedrock.

In [ ]:
# Initialize the base Bedrock model
base_model = BedrockModel(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0")

print("✓ Base model initialized:", base_model.get_config()["model_id"])

### Step 3: Build Semantic Similarity Model Wrapper

Now we'll create a custom model wrapper that:
1. **Calls the base model multiple times in parallel** (3x in this example)
2. **Computes semantic similarity** between all candidate responses
3. **Consolidates to the best response** or raises an alert if inconsistency is detected

This wrapper extends `ParallelModelWithConsolidation` from the `hallucination_utils` module, which handles:
- Parallel execution of multiple model calls
- Stream processing and response collection
- Telemetry integration for observability

We just need to implement the `consolidate()` method with our semantic similarity logic.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class SemanticSimilarityModel(ParallelModelWithConsolidation):
    """
    Custom model wrapper that detects hallucinations via semantic similarity analysis.

    This model:
    - Generates N candidate responses in parallel
    - Computes pairwise semantic similarity between all candidates
    - Intervenes if mean similarity falls below threshold
    - Returns the first response if all candidates are consistent
    """

    def __init__(
        self, base_model, num_candidates: int = 3, similarity_threshold: float = 0.75
    ):
        """
        Args:
            base_model: The Strands model to wrap (e.g., BedrockModel)
            num_candidates: Number of parallel responses to generate
            similarity_threshold: Minimum mean similarity score (0-1) to pass
        """
        # Create list of models (repeat the same model N times)
        super().__init__([base_model] * num_candidates)
        self.similarity_threshold = similarity_threshold
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2")

    def consolidate(
        self, alternatives: list[ModelResponse]
    ) -> ModelResponse | tuple[ModelResponse, TraceAttributes]:
        """
        Analyze semantic similarity between candidate responses.

        Returns:
            Consolidated response with telemetry attributes
        """
        # Extract response texts from alternatives
        response_texts = [
            alt.message["content"][0]["text"]
            for alt in alternatives
            if "content" in alt.message and len(alt.message["content"]) > 0
        ]

        if len(response_texts) < 2:
            # Not enough responses to compare - return first one
            return (
                ModelResponse.from_alternatives(
                    alternatives=alternatives, final_message=alternatives[0].message
                ),
                {
                    "gen_ai.guardrail.semantic_similarity.intervened": False,
                    "gen_ai.guardrail.semantic_similarity.status": "insufficient_samples",
                },
            )

        # Step 1: Encode all responses to embeddings
        embeddings = self.encoder.encode(response_texts)

        # Step 2: Calculate pairwise similarities
        similarity_matrix = cosine_similarity(embeddings)

        # Step 3: Get upper triangle (excluding diagonal)
        mask = np.triu(np.ones_like(similarity_matrix, dtype=bool), k=1)
        similarities = similarity_matrix[mask]

        # Step 4: Calculate metrics
        mean_similarity = float(np.mean(similarities))
        min_similarity = float(np.min(similarities))
        max_similarity = float(np.max(similarities))
        std_similarity = float(np.std(similarities))

        # Step 5: Determine if we should intervene
        intervened = mean_similarity < self.similarity_threshold

        # Step 6: Prepare telemetry attributes
        trace_attrs = {
            "gen_ai.guardrail.semantic_similarity.intervened": intervened,
            "gen_ai.guardrail.semantic_similarity.mean_score": mean_similarity,
            "gen_ai.guardrail.semantic_similarity.min_score": min_similarity,
            "gen_ai.guardrail.semantic_similarity.max_score": max_similarity,
            "gen_ai.guardrail.semantic_similarity.std_score": std_similarity,
            "gen_ai.guardrail.semantic_similarity.threshold": self.similarity_threshold,
            "gen_ai.guardrail.semantic_similarity.num_candidates": len(alternatives),
        }

        if intervened:
            # Hallucination detected - you could raise an error, return a warning, etc.
            trace_attrs["gen_ai.guardrail.semantic_similarity.status"] = (
                "INCONSISTENT_RESPONSES"
            )
            print(f"⚠️ Semantic Similarity Guardrail: Inconsistent responses detected!")
            print(
                f"   Mean similarity: {mean_similarity:.3f} (threshold: {self.similarity_threshold})"
            )
        else:
            trace_attrs["gen_ai.guardrail.semantic_similarity.status"] = (
                "CONSISTENT_RESPONSES"
            )
            print(f"✓ Semantic Similarity Guardrail: Responses are consistent")
            print(f"   Mean similarity: {mean_similarity:.3f}")

        # Return first response as the consolidated output
        # (In production, you might choose the response with highest avg similarity to others)
        return (
            ModelResponse.from_alternatives(
                alternatives=alternatives, final_message=alternatives[0].message
            ),
            trace_attrs,
        )


print("✓ SemanticSimilarityModel class defined")

### Step 4: Instantiate the Semantic Similarity Model

Now let's create an instance of our guardrail-enabled model.

In [ ]:
# Create semantic similarity model with 3 parallel candidates
semsim_model = SemanticSimilarityModel(
    base_model=base_model,
    num_candidates=3,
    similarity_threshold=0.75,  # Adjust based on your requirements
)

print("✓ Semantic Similarity Model created")
print(f"  - Number of parallel candidates: 3")
print(f"  - Similarity threshold: 0.75")
print(f"  - Encoder model: all-MiniLM-L6-v2")

### Step 5: Create the Strands Agent

Finally, let's create a Strands agent that uses our semantic similarity model.

In [ ]:
# Create agent with our custom model
agent = Agent(model=semsim_model)

print("✓ Agent created and ready to use!")
print("\nThe agent will:")
print("  1. Generate 3 candidate responses in parallel for each query")
print("  2. Compute semantic similarity between all candidates")
print("  3. Alert if responses are inconsistent (potential hallucination)")
print("  4. Return the most reliable response")

### Step 6: Test the Agent

Let's test our agent with different types of questions to see the semantic similarity guardrail in action.

#### Test 1: Factual Question (Should be Consistent)

In [ ]:
print("=" * 70)
print("TEST 1: Factual Question")
print("=" * 70)

result = agent("What is the capital of France?")
print("\nFinal Response:", result.message["content"][0]["text"])

In [ ]:
def evaluate_with_semantic_similarity(
    prompt: str,
    num_candidates: int = 3,
    similarity_threshold: float = 0.75,
    show_details: bool = True,
) -> dict:
    """
    Evaluate a prompt using semantic similarity-based hallucination detection.

    Args:
        prompt: The question/prompt to evaluate
        num_candidates: Number of parallel responses to generate
        similarity_threshold: Minimum similarity score to pass (0-1)
        show_details: Whether to print detailed analysis

    Returns:
        Dictionary with response and detection metrics
    """
    # Create model and agent
    model = SemanticSimilarityModel(
        base_model=base_model,
        num_candidates=num_candidates,
        similarity_threshold=similarity_threshold,
    )
    eval_agent = Agent(model=model)

    if show_details:
        print(f"\n{'=' * 70}")
        print(f"SEMANTIC SIMILARITY EVALUATION")
        print(f"{'=' * 70}")
        print(f"Prompt: {prompt}")
        print(f"Candidates: {num_candidates} | Threshold: {similarity_threshold}")
        print(f"{'=' * 70}\n")

    # Run the agent
    result = eval_agent(prompt)

    # Extract response
    response_text = result.message["content"][0]["text"]

    if show_details:
        print(f"\n{'=' * 70}")
        print(f"FINAL RESPONSE:")
        print(f"{'=' * 70}")
        print(response_text)
        print(f"{'=' * 70}")

    return {
        "prompt": prompt,
        "response": response_text,
        "num_candidates": num_candidates,
        "threshold": similarity_threshold,
        "result": result,
    }


# Example usage
evaluation = evaluate_with_semantic_similarity(
    "What are the main causes of climate change?",
    num_candidates=3,
    similarity_threshold=0.75,
)

You can go to the Langfuse console to investigate the detailed traces of the agent outputs.

![langfuse-lab2](./img/langfuse-lab2.png)

### Practical Wrapper Function

Here's a practical wrapper you can use to easily evaluate prompts with the semantic similarity agent:

### Production Deployment Considerations

When deploying this pattern to production, consider:

#### 1. Performance vs. Accuracy Tradeoffs
- **Latency**: Running 3 parallel model calls increases latency slightly, but reduces wall-clock time vs. sequential calls
- **Cost**: 3x model invocations = 3x cost per query
- **Optimization**: Use caching, reduce `num_candidates` for non-critical queries, or use faster/cheaper models

#### 2. Threshold Tuning
- **similarity_threshold=0.75** is a starting point - tune based on your domain
- Lower threshold (e.g., 0.6) = fewer false positives, but might miss hallucinations
- Higher threshold (e.g., 0.9) = stricter checking, but might flag legitimate variation
- Use A/B testing and human evaluation to find optimal threshold

#### 3. Error Handling
- Current implementation prints warnings but returns a response anyway
- In production, you might:
  - Raise an exception to force human review
  - Return a confidence score to the user
  - Fallback to a RAG system or web search
  - Log to monitoring system for offline analysis

#### 4. Observability
- Enable Langfuse or another observability platform
- Track metrics: `mean_similarity`, `intervention_rate`, `user_satisfaction`
- Set up alerts when intervention rate exceeds normal baselines
- Review flagged responses to improve prompts and thresholds

#### 5. Advanced Consolidation Strategies
- Current implementation returns the first response
- Alternatives:
  - Return the response with **highest average similarity** to others
  - Use **majority voting** for classification tasks
  - **Blend responses** for creative tasks
  - Apply **secondary checks** (e.g., NLI contradiction detection)

#### 6. Multi-Model Ensembles
- Instead of calling the same model 3x, call different models:
  ```python
  models = [
      BedrockModel("claude-haiku-4-5"),
      BedrockModel("claude-3-5-sonnet"),
      # Could even mix SageMaker models, etc.
  ]
  ensemble_model = SemanticSimilarityModel(models)
  ```
- Provides diversity and can catch model-specific hallucinations

#### 7. Integration with Other Guardrails
- Combine with **Bedrock Guardrails** for content filtering
- Add **contextual grounding** checks if using RAG
- Layer with **Token Probability Level methods** (perplexity, entropy) for more robust detection

<br/>

> **Key Considerations**
>
> These methods have higher latency and computational cost due to multiple model generations and additional processing. However, Section 6 demonstrates how to mitigate this through parallel execution and production optimization strategies. These methods are valuable for high-stakes applications where accuracy is critical, and can be combined with faster methods for different use cases.

## 7. Conclusion and Best Practices
(<a href="#top">Go to top</a>)

### What We've Learned

In this notebook, we've explored three powerful Response Level techniques for detecting LLM hallucinations:

1. **Semantic Similarity Detection** - Fast and effective for detecting response consistency
2. **Non-Contradiction Probability** - Excellent for identifying logical inconsistencies  
3. **Normalized Semantic Negentropy** - Most sophisticated, measuring semantic uncertainty
<br>

| Method | What It Detects | When to Use (High-Level Context) | Why It Works |
|--------|----------------|----------------------------------|--------------|
| **Semantic Similarity Detection** | Inconsistent knowledge across multiple generations | • **Content verification** where consistency matters<br>• **Knowledge validation** with available compute budget<br>• **Quick consistency checks** for important claims | True knowledge should be expressed the same way repeatedly - if answers vary wildly, the model isn't really "sure" |
| **Non-Contradiction Probability** | Logical inconsistencies between responses | • **Factual content** where contradictions are unacceptable<br>• **Research applications** requiring logical coherence<br>• **Decision support** systems with clear right/wrong answers | Real facts don't contradict themselves - if multiple responses conflict, at least one must be wrong |
| **Normalized Semantic Negentropy** | Semantic uncertainty through response clustering | • **High-precision applications** requiring confidence scores<br>• **Research environments** with computational resources<br>• **Advanced quality control** for critical outputs | Confident knowledge produces similar responses that cluster together - scattered responses indicate uncertainty |

### Key Takeaways

- **No single method is perfect** - Different techniques catch different types of hallucinations
- **Model quality varies significantly** - Some models are much more consistent than others
- **Question type matters** - Factual questions and creative prompts require different approaches
- **Confidence thresholds are domain-specific** - Adjust based on your use case criticality

### Best Practices for Production

1. **Use Multiple Methods**: Combine techniques for more robust detection
2. **Set Appropriate Thresholds**: Adjust confidence levels based on domain requirements
3. **Consider Computational Cost**: Balance accuracy with response time needs
4. **Monitor and Adapt**: Continuously evaluate and update detection parameters
5. **Human-in-the-Loop**: Always have escalation paths for uncertain cases

### Next Steps

- **Integrate with existing systems**: Add hallucination detection to your AI pipelines
- **Build domain-specific datasets**: Create evaluation sets for your specific use cases
- **Experiment with ensemble methods**: Combine multiple models and detection techniques
- **Explore fine-tuning**: Train models to be more consistent in your domain

### Additional Resources

- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Sentence Transformers Documentation](https://www.sbert.net/)
- [HuggingFace Transformers Guide](https://huggingface.co/docs/transformers)

---

**Remember**: The goal isn't to eliminate all uncertainty, but to **quantify and manage it** appropriately for your specific use cases.

Happy detecting!

## 8. (Optional) Challenge Exercises
(<a href="#top">Go to top</a>)

<div style="border: 4px solid coral; text-align: left; margin: auto; padding: 20px;">
    <h3>🎯 Try These Exercises on your own time!</h3>
    <p><b>Complete these challenges to deepen your understanding of hallucination detection:</b></p>
    <h4>Beginner Challenges:</h4>
    <ol>
        <li><b>Different Question Types:</b> Test the methods with various question types:
            <ul>
                <li>Historical facts: "When did World War II end?"</li>
                <li>Scientific concepts: "What is photosynthesis?"</li>
                <li>Fictional scenarios: "What is Sherlock Holmes' favorite tea?"</li>
            </ul>
        </li>
        <li><b>Model Comparison:</b> Compare all available models on the same question</li>
        <li><b>Parameter Tuning:</b> Experiment with different temperature values and sample sizes</li>
    </ol>
    <h4>Intermediate Challenges:</h4>
    <ol start="4">
        <li><b>Custom Thresholds:</b> Modify the confidence thresholds in each method</li>
        <li><b>Ambiguous Questions:</b> Test with deliberately ambiguous prompts</li>
        <li><b>Method Combination:</b> Create a combined score using all three methods</li>
    </ol>
    <h4>Advanced Challenges:</h4>
    <ol start="7">
        <li><b>Domain-Specific Testing:</b> Test on medical, legal, or technical questions</li>
        <li><b>Multi-Language Support:</b> Adapt methods for non-English responses</li>
        <li><b>Real-Time Application:</b> Build a web interface for live hallucination detection</li>
    </ol>
</div>